In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import gc
import lightgbm as lgb
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge
from joblib import Parallel, delayed
import gc
import seaborn as sns
from scipy import stats
import time 
from joblib import Parallel, delayed
from sklearn.manifold import TSNE
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier, NeighborhoodComponentsAnalysis, KNeighborsRegressor, LocalOutlierFactor
from sklearn.cluster import AgglomerativeClustering

import plotly.io as plt_io
import plotly.graph_objects as go



In [ ]:
%%time
details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
details.sort_values(by=['Asset_ID'], inplace = True)
print(details)

def read_csv_strict(file_name='/kaggle/input/g-research-crypto-forecasting/train.csv'):
    df = pd.read_csv(file_name)
    
    df2 = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')

    df = pd.concat([df,df2])
    
    df.sort_values(['Asset_ID','timestamp'],ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['Weight'] = df.Asset_ID.map(details.Weight)
    
    
    df['VWAP'] = np.where(np.isinf(df.VWAP),df.Close,df.VWAP)

        
    df['date_time'] = pd.to_datetime(df['timestamp'], unit='s')
    df_tr = df[df['date_time'] > '2021-01-01 00:00:00']
    df_ts = df[df['date_time'] >= '2021-06-13 00:00:00']
        
    return df_tr, df_ts
df, df_ts = read_csv_strict()

del read_csv_strict
df.tail(20)

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace = True, drop = True)
df

In [ ]:
df_idx_timestamp = df.pivot(index='timestamp', columns='Asset_ID', values='Target')
df_idx_Asset_ID = df.pivot(index='Asset_ID', columns='timestamp', values='Target')
df_idx_timestamp

In [ ]:
df_idx_timestamp.dropna(inplace=True)
df_idx_timestamp.reset_index(inplace = True, drop = True)
df_idx_Asset_ID = df_idx_timestamp.T

In [ ]:
neigh = NearestNeighbors(n_neighbors=6)
nbrs = neigh.fit(df_idx_Asset_ID)
neighb = nbrs.kneighbors(df_idx_Asset_ID,  return_distance=False)

neighb
for i,x in enumerate(df_idx_Asset_ID.index):
    print(neighb[i][0], neighb[i][1], neighb[i][2],neighb[i][3],neighb[i][4],neighb[i][5])